## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

# Sources
extracting frames from video:  https://stackoverflow.com/questions/33311153/python-extracting-and-saving-video-frames
a.	Real time lane detection for autonomous vehicles, Assidiq et. al.
b.	Saad Bedros, Hough Transform and Thresholding lecture, University of Minnesota 
c.	Lane detection techniques review, Kaur and Kumar
d.	An Adaptive Method for Lane Marking Detection Based on HSI Color Model, Tran and Cho
e.	LANE CHANGE DETECTION AND TRACKING FOR A SAFE-LANE APPROACH IN REAL TIME VISION BASED NAVIGATION SYSTEMS, Somasundaram, Ramachandran, Kavitha
f.	A Robust Lane Detection and Departure Warning System, Mrinal Haloi and Dinesh Babu Jayagopi
g.	Steerable filters
h.	A layered approach to robust lane detection at night, Hayes and Pankati
i.	SHADOW DETECTION USING COLOR AND EDGE INFORMATION

In [1]:
#command line functions
#os.rmdir('../Undistorted Test Images')
#os.mkdir('../Undistorted_Test_Images')
#os.remove('../overpass.mp4')
#os.remove('../pavement.mp4')
#os.remove('../leaves.mp4')
#os.remove('../shadows.mp4')
#os.remove('../test_images/undistorted_straight_lines2.jpg')

Notes:
grayscale - doesn't do well on bright roads.  I tried using red instead.
magnitude - does great on black road, even way out to a distance, disappears on light color road
yellow with s and h - works well, but not out to a distance, even on changing road and can't handle shadows
white with l, s, and r - almost as good as magnitude on black roads, much better on imperfect roads
shadows - sobel_y doesn't do so well, but sobel_x and magnitude are pretty good 

# Import libraries

In [14]:
import numpy as np
import os
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt
from moviepy.editor import VideoFileClip
from IPython.display import HTML



# Helper functions

In [243]:
def cal_undistort(img, objpoints, imgpoints):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist


def threshold(image, thresh_min=0, thresh_max=255, scale = True):
    if scale:
        scaled = np.uint8(255*image/np.max(image)) # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    else:
        scaled = image
    binary_output = np.zeros_like(scaled)
    # Masking for region of interest
    mask = np.zeros_like(scaled)   
    ignore_mask_color = 100   
    imshape = scaled.shape
    vertices = np.array([[(0,660),(0, 420), (imshape[1], 420), (imshape[1],660)]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    
    binary_output[(scaled >= thresh_min) & (scaled <= thresh_max) & (mask > 0)] = 1
    return binary_output

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    # 2) Take the gradient in x and y separately
    # 3) Take the absolute value of the x and y gradients
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    # 5) Create a binary mask where direction thresholds are met
    # 6) Return this mask as your binary_output image
    gray = image[:,:,0]#cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # 1) Convert to grayscale
    sobel_x = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0,ksize=sobel_kernel))
    sobel_y = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1,ksize=sobel_kernel))
    dir_grad = np.absolute(np.arctan2(sobel_y,sobel_x))
    print(np.amin(dir_grad))
    print(np.amax(dir_grad))
    #scaled_sobel = np.uint8(255*dir_grad/np.max(dir_grad)) # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    binary_output = np.zeros_like(dir_grad)
    
    # Masking for region of interest
    mask = np.zeros_like(dir_grad)   
    ignore_mask_color = 100   
    imshape = dir_grad.shape
    vertices = np.array([[(0,660),(0, 420), (imshape[1], 420), (imshape[1],660)]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
        
    binary_output[(dir_grad >= thresh[0]) & (dir_grad <= thresh[1]) & (mask > 0)] = 1
    return binary_output

# Create test images from the challenge videos

In [19]:
overpass = VideoFileClip("./challenge_video.mp4").subclip(4.2,4.3)
overpass.write_videofile('test_videos/overpass.mp4', audio=False)
pavement = VideoFileClip("./challenge_video.mp4").subclip(6,6.1)
pavement.write_videofile('test_videos/pavement.mp4', audio=False)
leaves = VideoFileClip("./harder_challenge_video.mp4").subclip(3,3.1)
leaves.write_videofile('test_videos/leaves.mp4', audio=False)
shadows = VideoFileClip("./harder_challenge_video.mp4").subclip(7,7.1)
shadows.write_videofile('test_videos/shadows.mp4', audio=False)

names =['overpass', 'pavement', 'leaves', 'shadows']
for fname in names:
    vidcap = cv2.VideoCapture('test_videos/%s.mp4' %(fname))
    print('reading image')
    success,image = vidcap.read()
    count = 0
    success = True
    while success:
      cv2.imwrite('test_images/%s_frame%d.jpg' %(fname, count), image)     
      success,image = vidcap.read()
      print('Read a new frame: ', success)
      count += 1


[MoviePy] >>>> Building video test_videos/overpass.mp4
[MoviePy] Writing video test_videos/overpass.mp4


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.53it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/overpass.mp4 

[MoviePy] >>>> Building video test_videos/pavement.mp4
[MoviePy] Writing video test_videos/pavement.mp4


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.72it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/pavement.mp4 

[MoviePy] >>>> Building video test_videos/leaves.mp4
[MoviePy] Writing video test_videos/leaves.mp4


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.53it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/leaves.mp4 

[MoviePy] >>>> Building video test_videos/shadows.mp4
[MoviePy] Writing video test_videos/shadows.mp4


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.85it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/shadows.mp4 

reading image
Read a new frame:  True
Read a new frame:  True
Read a new frame:  False
reading image
Read a new frame:  True
Read a new frame:  True
Read a new frame:  False
reading image
Read a new frame:  True
Read a new frame:  True
Read a new frame:  False
reading image
Read a new frame:  True
Read a new frame:  True
Read a new frame:  False


# First, I'll compute the camera calibration using chessboard images

In [43]:

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        plt.imshow(img)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)


# TODO: Write a function that takes an image, object points, and image points
# performs the camera calibration, image distortion correction and 
# returns the undistorted image
img = cv2.imread('camera_cal/calibration1.jpg')
undistorted = cal_undistort(img, objpoints, imgpoints)
cv2.imwrite('camera_cal/undistorted.jpg', undistorted)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undistorted)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

print('done')


done


# Input and undistort raw images

In [39]:
file_list = os.listdir("test_images/")
#fig=plt.figure(figsize=(10, 80))
i = 1

for name in file_list:
    image = cv2.imread('test_images/%s'  %(name))   #read in the image
    print('reading %s' %(name))
    undistorted = cal_undistort(image, objpoints, imgpoints)
    cv2.imwrite('Undistorted_Test_Images/undistorted_%s'  %(name), undistorted)
    
b,g,r = cv2.split(image)       # get b,g,r
rgb_image = cv2.merge([r,g,b])     # switch it to rgb
b,g,r = cv2.split(undistorted)       # get b,g,r
rgb_undistorted = cv2.merge([r,g,b])     # switch it to rgb


f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(rgb_image)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(rgb_undistorted)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
print('done')
#gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY) # grayscale the image
#blur_gray = gaussian_blur(gray, 5)  #add gaussian blur
#edges = cv2.Canny(blur_gray, 50, 150)  # add canny 

reading leaves_frame0.jpg
reading leaves_frame1.jpg
reading leaves_frame2.jpg
reading overpass_frame0.jpg
reading overpass_frame1.jpg
reading overpass_frame2.jpg
reading pavement_frame0.jpg
reading pavement_frame1.jpg
reading pavement_frame2.jpg
reading shadows_frame0.jpg
reading shadows_frame1.jpg
reading shadows_frame2.jpg
reading straight_lines1.jpg
reading straight_lines2.jpg
reading test1.jpg
reading test2.jpg
reading test3.jpg
reading test4.jpg
reading test5.jpg
reading test6.jpg
done


Use color transforms, gradients, etc., to create a thresholded binary image.

In [263]:
file_list = [3,4,5]
sobel_kernel = 5
chart=plt.figure(figsize=(24, 12))

for name in file_list:
    print('reading %s' %(name))
    image = mpimg.imread('Undistorted_Test_Images/undistorted_test%s.jpg'  %(name))   #read in the image
    
    gray =  image[:,:,0]#cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) # 1) Convert to grayscale
    blur_gray = gaussian_blur(gray, 5)  #add gaussian blur
    sobel_x = np.absolute(cv2.Sobel(blur_gray, cv2.CV_64F, 1, 0,ksize=sobel_kernel))
    sobel_y = np.absolute(cv2.Sobel(blur_gray, cv2.CV_64F, 0, 1,ksize=sobel_kernel))
    mag_grad = np.sqrt(np.power(sobel_x,2)+np.power(sobel_y,2))
    # = np.arctan2(sobel_y,sobel_x)
    R = image[:,:,0]
    G = image[:,:,1]
    B = image[:,:,2]
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    
    x_binary = threshold(sobel_x, 20, 60)
    y_binary = threshold(sobel_y, 20, 60)
    mag_binary = threshold(mag_grad, 35, 150)                      
    Canny_binary = cv2.Canny(gray, 50, 150)  # add canny
    dir_binary = dir_threshold(image, sobel_kernel=3, thresh=(1,1.6))
    dir_interesting = ((Canny_binary > 0) & (dir_binary > 0) & (x_binary > 0))
    r_binary = threshold(R, 200,255, False)
    h_binary = threshold(H, 20, 100, False)
    s_binary = threshold(S, 90, 255, False)
    l_binary = threshold(L, 200, 255, False)

    
    #combining HLS colorspace to identify white and yellow lines.  I got inspiration from the HLS 
    #color thresholds lesson and from Tran's paper referenced above
    white = (l_binary & (s_binary | r_binary))
    yellow = (s_binary & h_binary)
    combined = (white | yellow | dir_interesting)
    
    chart.add_subplot(3, 2, (name-2)*2-1)
    plt.imshow(image)
    chart.add_subplot(3, 2, (name-2)*2)
    plt.imshow(combined)

    
    
f, axes = plt.subplots(4, 2, figsize=(24, 12))
f.tight_layout()

axes[0,0].imshow(image)
axes[0,0].set_title('Original Image', fontsize=20)

axes[0,1].imshow(gray)
axes[0,1].set_title('Grayscale', fontsize=20)

axes[1,0].imshow(blur_gray)
axes[1,0].set_title('Blurred Grayscale', fontsize=20)

axes[1,1].imshow(x_binary)
axes[1,1].set_title('SobelX Binary', fontsize=20)

axes[2,0].imshow(y_binary)
axes[2,0].set_title('SobelY Binary', fontsize=20)

axes[2,1].imshow(mag_binary)
axes[2,1].set_title('Magnitude Binary', fontsize=20)

axes[3,0].imshow(dir_interesting)
axes[3,0].set_title('Direction Binary', fontsize=20)

axes[3,1].imshow(Canny_binary)
axes[3,1].set_title('Canny Binary', fontsize=20)


fig, color_axes = plt.subplots(3, 2, figsize=(24, 9))
fig.tight_layout()

color_axes[0,0].imshow(S)
color_axes[0,0].set_title('S', fontsize=20)

color_axes[0,1].imshow(s_binary)
color_axes[0,1].set_title('S binary', fontsize=20)

color_axes[1,0].imshow(H)
color_axes[1,0].set_title('H', fontsize=20)

color_axes[1,1].imshow(h_binary)
color_axes[1,1].set_title('H binary', fontsize=20)

color_axes[2,0].imshow(L)
color_axes[2,0].set_title('L', fontsize=20)

color_axes[2,1].imshow(l_binary)
color_axes[2,1].set_title('L binary', fontsize=20)

fig2, axes = plt.subplots(2, 2, figsize=(24, 9))
fig2.tight_layout()

axes[0,0].imshow(white)
axes[0,0].set_title('White Lines', fontsize=20)

axes[0,1].imshow(yellow)
axes[0,1].set_title('Yellow Lines', fontsize=20)

axes[1,0].imshow(image)
axes[1,0].set_title('Original', fontsize=20)

axes[1,1].imshow(combined)
axes[1,1].set_title('Combined', fontsize=20)


reading 3
0.0
1.5707963267948966
reading 4
0.0
1.5707963267948966
reading 5
0.0
1.5707963267948966


Text(0.5,1,'Combined')